In [3]:
# DEC. 2022
# Pipeline by Zhilin Ning

# Load libraries
#.libPaths('/home/zlning/R/lib/')
library(scater)
library(Seurat)
library(tidyverse)
library(cowplot)
library(Matrix.utils)
library(edgeR)
library(dplyr)
library(magrittr)
library(Matrix)
library(purrr)
library(reshape2)
library(S4Vectors)
library(tibble)
library(SingleCellExperiment)
library(pheatmap)
library(apeglm)
library(png)
library(DESeq2)
library(RColorBrewer)
library(hdf5r)
library(ggpubr)
library(lme4)
library(ggthemes)
library(GEOquery) 

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    

In [2]:
gset<-getGEO(filename='GSE183904_series_matrix.txt.gz',getGPL=F)

In [25]:
meta <- data.frame(geo_accession = gset$geo_accession, title = gset$title, tissue = gset$characteristics_ch1)

In [30]:
filelist <- paste0(meta$geo_accession, "_", sapply(strsplit(meta$title,":"),'[[',1), ".csv.gz")

In [41]:
gz <- gzfile(filelist[1])

In [42]:
a <- read.csv(gz, head=T, row.names=1)

In [44]:
meta.1 <- data.frame(filename = filelist[1], barcode = colnames(a))

In [45]:
for(i in 2:length(filelist)){
    gz <- gzfile(filelist[i])
    b <- read.csv(gz, head=T, row.names=1)
    meta.tmp <- data.frame(filename = filelist[i], barcode = colnames(b))
    meta.1 <- rbind(meta.1, meta.tmp)
    a <- cbind(a,b)
}

In [49]:
meta$filelist <- filelist

In [73]:
tissue.tmp <- strsplit(sapply(strsplit(sapply(strsplit(meta$tissue, ":"), '[[',2),'\\('),'[[',1) , ' ')

In [82]:
meta$Tissue <- as.vector(factor(sapply(tissue.tmp, '[[', 2), levels = c('Primary', 'Peritonium'), labels = c("Primary_Gastric_Tissue", "Peritonium_Tissue")))

In [93]:
meta$Group <- sapply(strsplit(sapply(strsplit(sapply(strsplit(meta$tissue, ":"), '[[',2),'\\('),'[[',2) , ')'),'[[',1)

In [97]:
meta$sample <- sapply(strsplit(meta$title, ":"), '[[',1)

In [102]:
meta.s <- meta[match(meta.1$filename, meta$filelist),]

In [105]:
meta.2 <- data.frame(meta.1$barcode, meta.s$sample, meta.s$geo_accession, meta.s$Tissue, meta.s$Group)

In [107]:
colnames(meta.2) <- c("barcode","sample","geo_accession","Tissue","Group")

In [109]:
#counts <- Read10X(data.dir = dir)
#class(counts)
#[1] "dgCMatrix"
#attr(,"package")
#[1] "Matrix"

counts = a
scRNA <- CreateSeuratObject(counts = counts)
scRNA
#An object of class Seurat 
#33694 features across 9112 samples within 1 assay 
#Active assay: RNA (33694 features, 0 variable features)

An object of class Seurat 
26571 features across 158641 samples within 1 assay 
Active assay: RNA (26571 features, 0 variable features)

In [111]:
dataan <- as(as.matrix(counts), "dgCMatrix")


In [118]:
rownames(meta.2) <- meta.2$barcode

In [112]:
data.object <- CreateSeuratObject(counts = dataan, project = "GastricCancer", min.cells = 3, min.features = 200)

In [119]:
#head(meta.T[,1])

#row.names(meta.T) <- meta.T[,1]
#meta <- as.matrix(meta.T)
#head(meta)
#row.names(meta) <- meta[,1]
data.object <- AddMetaData(object = data.object, metadata = meta.2)

In [121]:
#getwd()
# Save final R object
write_rds(data.object,
          path = paste0("./data/seurat_labelled.rds"))
#write_rds(conserved_markers,
#          path = paste0("./",cancertype,".conserved_markers.rds"))    
 

Warning message:
“The `path` argument of `write_rds()` is deprecated as of readr 1.4.0.
ℹ Please use the `file` argument instead.”


In [123]:
data.object.1 <- subset(data.object, subset = Tissue == 'Peritonium_Tissue')

In [124]:
write_rds(data.object,
          path = paste0("./data/Peritonium_Tissue.seurat_labelled.rds"))

In [125]:
data.object.1 <- subset(data.object, subset = Tissue == 'Primary_Gastric_Tissue')

In [126]:
write_rds(data.object,
          path = paste0("./data/Primary_Gastric_Tissue.seurat_labelled.rds"))

# Cluster

In [ ]:
seurat_integrated <- readRDS("./data/Primary_Gastric_Tissue_seurat_labelled.rds")
seurat_integrated = subset(seurat_integrated, Group == 'Tumor')
seurat_integrated[["percent.mt"]] <- PercentageFeatureSet(seurat_integrated, pattern = "^MT-")
VlnPlot(seurat_integrated, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)
Immune <- subset(seurat_integrated, subset = nFeature_RNA > 500 & nFeature_RNA < 3000 & percent.mt < 15)
Dimension = 'harmony'
Immune <- NormalizeData(Immune, normalization.method = "LogNormalize", scale.factor = 10000)
Immune <- Seurat::FindVariableFeatures(Immune, selection.method = "vst", nfeatures = 3000) %>% 
    ScaleData(verbose = FALSE) %>% 
    RunPCA(pc.genes = Immune@var.genes, npcs = 20, verbose = FALSE)

Immune <- Immune %>% 
    RunHarmony("sample", plot_convergence = TRUE)

Immune <- Immune %>% 
    RunUMAP(reduction = Dimension, dims = 1:20) %>% 
    FindNeighbors(reduction = Dimension, dims = 1:20) %>% 
    FindClusters(resolution = 0.2)

options(repr.plot.height = 6, repr.plot.width = 8)
DimPlot(Immune, reduction = "umap", group.by = "seurat_clusters", pt.size = .1)

saveRDS(Immune, file = "./data/RawData.rds")
SaveH5Seurat(Immune, filename = "./data/RawData")
Convert("./data/RawData.h5seurat", dest = "h5ad")

In [3]:
# Cluster
##  0: T&NK
##  1: T&NK
##  2: PB
##  3: Macro
##  4: Epi
##  5: Fibroblast
##  6: B
##  7: Endothelia
##  8: Mast Cell
##  9: Pericyte
##  10:T&NK
##  11:PB
##  12:Epi

# 创建映射
cluster_mapping <- c("0" = "T&NK", 
                     "1" = "T&NK",
                     "2" = "PB",
                     "3" = "Macro",
                     "4" = "Epi",
                     "5" = "Fibroblast",
                     "6" = "B",
                     "7" = "Endothelia",
                     "8" = "Mast Cell",
                     "9" = "Pericyte",
                     "10" = "T&NK",
                     "11" = "PB",
                     "12" = "Epi")

# 将 seurat_cluster 值转换为因子
seurat_clusters <- as.factor(Immune$seurat_clusters)

# 使用映射添加新列
Immune$cell_type <- as.character(cluster_mapping[as.character(seurat_clusters)])


In [ ]:
write.table(Immune@meta.data, './data/Processed.meta.txt', sep = '\t', quote = FALSE)
write.table(Immune@reductions$umap@cell.embeddings, './data/UMAP.txt', sep = '\t', quote = FALSE)


In [ ]:
Immune = readRDS("./data/RawData.rds")
Immune = subset(Immune, seurat_clusters %in% c(4,12))
Dimension = 'pca'
Immune <- NormalizeData(Immune, normalization.method = "LogNormalize", scale.factor = 10000)
Immune <- Seurat::FindVariableFeatures(Immune, selection.method = "vst", nfeatures = 2000) %>% 
    ScaleData(verbose = FALSE) %>% 
    RunPCA(pc.genes = Immune@var.genes, npcs = 20, verbose = FALSE)
#Immune <- Immune %>% 
#    RunHarmony("sample", plot_convergence = TRUE)
Immune <- Immune %>% 
    RunUMAP(reduction = Dimension, dims = 1:20) %>% 
    FindNeighbors(reduction = Dimension, dims = 1:20) %>% 
    FindClusters(resolution = 0.1)
options(repr.plot.height = 6, repr.plot.width = 8)
DimPlot(Immune, reduction = "umap", group.by = "seurat_clusters", pt.size = .1)
write.table(Immune@meta.data, './data/Epi.meta.txt', sep = '\t', quote = FALSE)



Immune = readRDS("./data/RawData.rds")
Immune = subset(Immune, seurat_clusters %in% c(0,1,10))
Dimension = 'harmony'
Immune <- NormalizeData(Immune, normalization.method = "LogNormalize", scale.factor = 10000)
Immune <- Seurat::FindVariableFeatures(Immune, selection.method = "vst", nfeatures = 2000) %>% 
    ScaleData(verbose = FALSE) %>% 
    RunPCA(pc.genes = Immune@var.genes, npcs = 20, verbose = FALSE)
Immune <- Immune %>% 
    RunHarmony("sample", plot_convergence = TRUE)
Immune <- Immune %>% 
    RunUMAP(reduction = Dimension, dims = 1:20) %>% 
    FindNeighbors(reduction = Dimension, dims = 1:20) %>% 
    FindClusters(resolution = 0.2)
options(repr.plot.height = 6, repr.plot.width = 8)
DimPlot(Immune, reduction = "umap", group.by = "seurat_clusters", pt.size = .1)
write.table(Immune@meta.data, './data/T&NK.meta.txt', sep = '\t', quote = FALSE)


Immune = readRDS("./data/RawData.rds")
Immune = subset(Immune, seurat_clusters == 3)
Dimension = 'harmony'
Immune <- NormalizeData(Immune, normalization.method = "LogNormalize", scale.factor = 10000)
Immune <- Seurat::FindVariableFeatures(Immune, selection.method = "vst", nfeatures = 2000) %>% 
    ScaleData(verbose = FALSE) %>% 
    RunPCA(pc.genes = Immune@var.genes, npcs = 20, verbose = FALSE)
Immune <- Immune %>% 
    RunHarmony("sample", plot_convergence = TRUE)
Immune <- Immune %>% 
    RunUMAP(reduction = Dimension, dims = 1:20) %>% 
    FindNeighbors(reduction = Dimension, dims = 1:20) %>% 
    FindClusters(resolution = 0.2)

options(repr.plot.height = 6, repr.plot.width = 8)
DimPlot(Immune, reduction = "umap", group.by = "seurat_clusters", pt.size = .1)
write.table(Immune@meta.data, './data/Macro&DC.meta.txt', sep = '\t', quote = FALSE)

